In [6]:
# Make necessary/useful imports here
import pandas as pd
import random
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.cm
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# Set pandas options to not truncate
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

# Mount data 
from google.colab import drive

drive.mount("/content/drive", force_remount=True)
root_path = 'drive/My Drive/Datathon Citizens/data/'



Mounted at /content/drive


In [11]:
n = 6009259 # number of records in file
s = 60000 # sample size (can/should be changed to your preference)

skip_list = sorted(random.sample(range(1,n+1),n-s))
may_df = pd.read_csv(root_path + "zip9_coded_201905_pv.csv", skiprows=skip_list, dtype={'zip5': str},engine='python')
demo_df = pd.read_csv(root_path + "zip9_demographics_coded_pv.csv")

ParserError: ignored

In [8]:
demo_df.head()

,zip5,zip9_code,age,household_count,person_count,homebuyers,first_homebuyers
0,1001,11466,50.365501,6,10,0,0
1,1001,28771,35.258247,5,7,0,0
2,1001,52911,46.101577,10,26,0,0
3,1001,81991,44.539054,13,19,0,0
4,1001,83391,57.670359,9,17,0,0


In [0]:
may_demo_merge = may_df.merge(demo_df,
                              how='inner',
                              on='zip9_code',
                              suffixes=('_may','_demo'),
                              validate='one_to_one')

In [0]:
majority_missing = ['mortgage2_limit','mortgage2_balance','mortgage3_limit','mortgage3_balance',
                    'mortgage4_limit','mortgage4_balance','mortgage5_limit','mortgage5_balance',
                    'homeequity1_limit','homeequity1_balance','homeequity2_limit','homeequity2_balance',
                    'homeequity3_limit','homeequity3_balance','homeequity4_limit','homeequity4_balance',
                    'homeequity5_limit','homeequity5_balance','total_homeequity_limit',
                    'total_homeequity_balance','homeequity1_loan_to_value']
# Remove exploration, secondary target, and other extra fields
other_removals = ['bankcard_balance_decile','first_homebuyers','zip5_sep','zip5_demo','zip9_code'] 
may_demo_merge_vars = may_demo_merge.columns.values.tolist()
to_keep = [i for i in may_demo_merge_vars if i not in majority_missing]
to_keep = [i for i in to_keep if i not in other_removals]
may_data_final = may_demo_merge[to_keep]

In [0]:
may_data_final.head()

,zip5_may,bankcard_limit,bankcard_balance,bankcard_trades,bankcard_util,total_revolving_limit,total_revolving_balance,total_revolving_trades,total_revolving_util,mortgage1_limit,mortgage1_balance,mortgage1_open,mortgage2_open,mortgage3_open,mortgage4_open,mortgage5_open,total_mortgage_limit,total_mortgage_balance,total_mortgage_trades,mortgage1_loan_to_value,homeequity1_open,homeequity2_open,homeequity3_open,homeequity4_open,homeequity5_open,total_homeequity_trades,autoloan_open,studentloan_open,bankcard_open,homeequity_open,mortgage_open,age,household_count,person_count,homebuyers
0,01001,22947.285714,1644.321429,2.321429,0.162426,29916.571429,3742.321429,3.857143,0.178453,113791.66667,96674.50000,0.428571,0.000000,0.0,0.0,0.0,113791.66667,96674.500000,0.428571,80.416667,0.071429,0.0,0.0,0.0,0.0,0.071429,0.107143,0.357143,0.785714,0.071429,0.428571,55.172380,11,14,0
1,01001,21878.769231,4946.807692,3.807692,0.172333,26698.076923,5530.653846,5.538462,0.161418,136530.83333,116297.41667,0.461538,0.000000,0.0,0.0,0.0,136530.83333,116297.416670,0.461538,83.666667,0.076923,0.0,0.0,0.0,0.0,0.076923,0.346154,0.153846,0.923077,0.076923,0.461538,48.869264,7,13,0
2,01001,24840.923077,3797.564103,3.153846,0.209186,33841.025641,4502.307692,6.205128,0.184633,112168.28571,82553.00000,0.358974,0.025641,0.0,0.0,0.0,127543.28571,95382.285714,0.384615,70.428571,0.205128,0.0,0.0,0.0,0.0,0.205128,0.256410,0.307692,0.897436,0.153846,0.358974,46.991827,13,20,0
3,01001,30843.929825,6556.947368,3.192982,0.198558,37005.508772,8287.684210,4.736842,0.204095,119406.30769,102755.11538,0.456140,0.070175,0.0,0.0,0.0,121552.13333,103095.833330,0.526316,86.269231,0.035088,0.0,0.0,0.0,0.0,0.035088,0.350877,0.280702,0.771930,0.035088,0.526316,52.311936,15,30,0
4,01001,40486.800000,2746.900000,4.133333,0.094579,46108.333333,4200.900000,6.333333,0.101976,173250.00000,136548.75000,0.133333,0.000000,0.0,0.0,0.0,173250.00000,136548.750000,0.133333,80.250000,0.200000,0.0,0.0,0.0,0.0,0.200000,0.400000,0.200000,0.933333,0.200000,0.133333,56.400257,7,15,0


In [0]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
imputer = IterativeImputer(estimator=RandomForestRegressor(),random_state=1000)

impute_columns = imputer.fit_transform(may_data_final[["bankcard_util","total_revolving_util","mortgage1_limit","mortgage1_balance","total_mortgage_limit","total_mortgage_balance","mortgage1_loan_to_value"]])

KeyboardInterrupt: ignored

In [0]:
data_final_apr["bankcard_util"]=impute_columns[:,0]
data_final_apr["total_revolving_util"]=impute_columns[:,1]
data_final_apr["mortgage1_limit"]=impute_columns[:,2]
data_final_apr["mortgage1_balance"]=impute_columns[:,3]
data_final_apr["total_mortgage_limit"]=impute_columns[:,4]
data_final_apr["total_mortgage_balance"]=impute_columns[:,5]
data_final_apr["mortgage1_loan_to_value"]=impute_columns[:,6]

In [0]:
may_data_final.loc[,"bankcard_util"] = impute_columns[:,0]
